# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 07**: Structured Streaming with Files 

**Date**: October 10th 2025

**Student Name**: Juan Bernardo Orozco Quirarte

**Professor**: Pablo Camarillo Ramirez

### Create Spark Session

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming (files)") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")
# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/10 07:07:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Create Schema Logs

In [2]:
from bernardoorozco.spark_utils import SparkUtils
from pyspark.sql.functions import col

logs_schema = SparkUtils.generate_schema([("user", "string"), ("timestamp", "string"),("endpoint", "string"),("status","int")])

### Read stream (json files) and filter by critical errors

In [3]:
critical_logs = spark.readStream \
            .format("json") \
            .schema(logs_schema) \
            .load("/opt/spark/work-dir/data/logs/") \
            .filter(col("status") >= 500)

### Stream Execution for **critical errors**

In [4]:
query_critical = critical_logs.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", False) \
    .start() 

query_critical.awaitTermination(30)

-------------------------------------------
Batch: 0
-------------------------------------------
+-----+--------------------+---------+------+
|user |timestamp           |endpoint |status|
+-----+--------------------+---------+------+
|Pablo|10/10/2025, 01:07:27|/api/data|500   |
+-----+--------------------+---------+------+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+--------------------+--------+------+
|user |timestamp           |endpoint|status|
+-----+--------------------+--------+------+
|Paola|10/10/2025, 01:07:30|/login  |502   |
+-----+--------------------+--------+------+

-------------------------------------------
Batch: 2
-------------------------------------------
+----+---------+--------+------+
|user|timestamp|endpoint|status|
+----+---------+--------+------+
+----+---------+--------+------+

-------------------------------------------
Batch: 3
-------------------------------------------
+----+---------+------

False

### Stop session

In [5]:
sc.stop()

25/10/10 07:07:58 ERROR MicroBatchExecution: Query [id = ccd09793-6140-49c1-ab18-f2b6f487d211, runId = c40eef2e-7779-44b2-b68a-be32e902a690] terminated with error
org.apache.spark.SparkException: [INTERNAL_ERROR] The Spark SQL phase optimization failed with an internal error. You hit a bug in Spark or the Spark plugins you use. Please, report this bug to the corresponding communities or vendors, and provide the full stack trace. SQLSTATE: XX000
	at org.apache.spark.SparkException$.internalError(SparkException.scala:107)
	at org.apache.spark.sql.execution.QueryExecution$.toInternalError(QueryExecution.scala:643)
	at org.apache.spark.sql.execution.QueryExecution$.withInternalError(QueryExecution.scala:656)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:278)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:804)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:277)
	at org.apache.spark.sql